# Load change requests (df) resume book (resume_book) dataframes.

In [18]:
import pandas as pd

csv_url = f"https://docs.google.com/spreadsheets/d/1IgOnbPhOoCRDBcTf9FIHwP54rHwcqSyKSJTE-XKNnJw/export?format=csv&gid=523778578"
df = pd.read_csv(csv_url)
df = df[df['Done?'] != 'yes']
df.head(2)

,Done?,Timestamp,Email Address,First Name,Last Name,Preferred Name,Pronouns,Email,"Do you want to add, update, or remove your resume?",Graduation Quarter,Graduation Year,Are you looking for an internship or full-time position?,What types of roles are you looking for?,Please copy/paste the text from your resume here. It is ok if it's not pretty! We just want your info to be easily searchable.,Upload Resume,Why do you want your resume removed?
780,NaN,9/30/2024 14:52:08,myan266@cs.washington.edu,Ming,Yan,NaN,NaN,myan266@uw.edu,I already have a resume in this book and want ...,Spring,2028.0,Internship,Software Development/Software Engineering,NaN,https://drive.google.com/open?id=1UYfRrEhCYwGZ...,NaN
781,NaN,9/30/2024 15:33:43,emiyosh@cs.washington.edu,Emi,Yoshikawa,Emi,she/her,emiyoshikawa.dev@gmail.com,I already have a resume in this book and want ...,Autumn,2024.0,Full time,Software Development/Software Engineering,OBJECTIVE\nEMI YOSHIKAWA\n+1(425)606-8565 ⋄ em...,https://drive.google.com/open?id=1duuA-4z_kHJ6...,NaN


In [19]:
print(f"There are {len(df)} updates to make.")
df['Do you want to add, update, or remove your resume?'].value_counts()

There are 222 updates to make.


,count
"Do you want to add, update, or remove your resume?",
I already have a resume in this book and want to update it to a newer version or update my information in the survey.,217
Add my first resume to this resume book,3
I am no longer looking for a position and wish to remove my resume.,2


In [20]:
csv_url = f"https://docs.google.com/spreadsheets/d/1xqvrDynnWfslrSnOymMtJCrMvmAQBka70L7i8USc5Bs/export?format=csv&gid=0"
resume_book = pd.read_csv(csv_url)
resume_book.head(2)

,First Name,Last Name,Preferred Name,Pronouns,Email,Grad Quarter,Grad Year,Are you looking for an internship or full-time position?,What types of roles are you looking for?,Resume Full Text,Resume Link
0,Ananya,Aatreya,NaN,she/her,ananyaa06@gmail.com,Spring,2028.0,Internship,"Software Development/Software Engineering, Pro...",Machine generated alternative text:\nAnanya Aa...,https://drive.google.com/open?id=1UXiJ5A-YId0y...
1,Abdalla,Abdalla,NaN,NaN,abdalla.abdalla3829@gmail.com,Spring,2026.0,Internship,"Software Development/Software Engineering, Use...",Abdalla Abdalla\n425-399-3922 | abdalla.abdall...,https://drive.google.com/open?id=1TPDqNh3kM_7q...


In [21]:
print(f"There are {len(resume_book)} resume entries.")

There are 696 resume entries.


# Clean dataframes

In [22]:
df_cols_to_clean = ['Email ', 'Email Address', "First Name", "Last Name"]
resume_cols_to_clean = ['Email', 'First Name', "Last Name"]

for col in df_cols_to_clean:
    if col in df.columns:
        df[col] = df[col].astype(str).str.lower().str.replace(r'\s+', '', regex=True)

for col in resume_cols_to_clean:
    if col in resume_book.columns:
        resume_book[col] = resume_book[col].astype(str).str.lower().str.replace(r'\s+', '', regex=True)

# Make updates

In [23]:
# remove resume
from tqdm import tqdm
remove_rows = df[df["Do you want to add, update, or remove your resume?"] == "I am no longer looking for a position and wish to remove my resume."]
print(f"Deleting rows for {remove_rows['Email ']}")

for index, row in tqdm(remove_rows.iterrows()):
    email = row["Email "]
    email2 = row['Email Address']

    if ((pd.isna(email) and pd.isna(email2)) or email == ""):
        raise ValueError(f"Email is missing for row {index} where resume removal is requested.")

    matching_rows = resume_book[(resume_book["Email"] == email) | (resume_book["Email"] == email2)]

    if len(matching_rows) > 1:
        print(f"Warning: Multiple rows found in resume_book with the email {email} or {email2}. Deleting all matching rows.")

    resume_book = resume_book[resume_book["Email"] != email]

    if len(matching_rows) == 0:
      # resort to name
      firstname = row['First Name']
      lastname = row['Last Name']
      matching_rows = resume_book[(resume_book["First Name"] == firstname) & (resume_book["Last Name"] == lastname)]

      if len(matching_rows) > 1:
        print(f"Warning: Multiple rows found in resume_book with the email {email} or {email2}. Abort deletion.")

      else:
        resume_book = resume_book[(resume_book["First Name"] != firstname) & (resume_book["Last Name"] != lastname)]

Deleting rows for 988    shawnc6@cs.washington.edu
992                 vperi@uw.edu
Name: Email , dtype: object


2it [00:00, 143.51it/s]


In [24]:
print(f"There are {len(resume_book)} resume entries.")

There are 694 resume entries.


In [25]:
df_update_cols = df.loc[:, "First Name":"Upload Resume"].columns.tolist()
df_update_cols = [col for col in df_update_cols if col != "Do you want to add, update, or remove your resume?"]

In [26]:
# add rows
add_rows = df[df["Do you want to add, update, or remove your resume?"] == "Add my first resume to this resume book"]
add_rows = add_rows[df_update_cols]
add_rows.columns = resume_book.columns

resume_book = pd.concat([resume_book, add_rows], ignore_index=True)
print(len(resume_book))

697


In [27]:
# update rows
update_rows = df[df["Do you want to add, update, or remove your resume?"] == "I already have a resume in this book and want to update it to a newer version or update my information in the survey."]
init_len = len(update_rows)
update_rows = update_rows.drop_duplicates(subset=["Email "], keep="last")
update_rows = update_rows.drop_duplicates(subset=["First Name", "Last Name"], keep="last")
print(f"{init_len - len(update_rows)} same-user updates removed. {len(update_rows)} updates to make.")

update_rows_new = update_rows[df_update_cols]
update_rows_new.columns = resume_book.columns

resume_book = pd.concat([resume_book, update_rows_new], ignore_index=True)
print(len(resume_book))

36 same-user updates removed. 181 updates to make.
878


In [28]:
for index, row in tqdm(update_rows.iterrows()):
    email = row["Email "]
    email2 = row['Email Address']

    if ((pd.isna(email) and pd.isna(email2)) or email == ""):
        raise ValueError(f"Email is missing for row {index} where resume removal is requested.")

    matching_rows = resume_book[(resume_book["Email"] == email) | (resume_book["Email"] == email2)]

    if len(matching_rows) > 1:
         later_matching_row_index = matching_rows.index.max()
         resume_book = resume_book.drop(matching_rows.index[matching_rows.index != later_matching_row_index])

    firstname = row['First Name']
    lastname = row['Last Name']

    matching_rows = resume_book[(resume_book["First Name"] == firstname) & (resume_book["Last Name"] == lastname)]

    if len(matching_rows) > 1:
         later_matching_row_index = matching_rows.index.max()
         resume_book = resume_book.drop(matching_rows.index[matching_rows.index != later_matching_row_index])

print(f"\nThere are {len(resume_book)} resume entries.")

181it [00:01, 112.62it/s]


There are 771 resume entries.


# Clean up

In [29]:
resume_book = resume_book.reset_index(drop=True)
resume_book['First Name'] = resume_book['First Name'].str.capitalize()
resume_book['Last Name'] = resume_book['Last Name'].str.capitalize()
# resume_book.to_csv('resume_book.csv', index=False)

In [30]:
csv_url = f"https://docs.google.com/spreadsheets/d/1IgOnbPhOoCRDBcTf9FIHwP54rHwcqSyKSJTE-XKNnJw/export?format=csv&gid=523778578"
df = pd.read_csv(csv_url)
df['Done?'] = 'yes'

# Port updates to resume_book gsheets

In [37]:
from google.colab import auth
import gspread
from google.auth.transport.requests import Request
from google.auth import default
from gspread_dataframe import set_with_dataframe

auth.authenticate_user()
creds, _ = default()
client = gspread.authorize(creds)

In [40]:
spreadsheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1xqvrDynnWfslrSnOymMtJCrMvmAQBka70L7i8USc5Bs/edit?gid=0#gid=0')
sheet = spreadsheet.get_worksheet(1)
sheet.clear()
set_with_dataframe(sheet, resume_book)
print("Sheet updated successfully.")


Sheet updated successfully.


In [41]:
import math
num_rows = math.ceil(len(resume_book) / 100) * 100
num_rows

800

In [42]:
from googleapiclient.discovery import build

auth.authenticate_user()
creds, _ = default()
client = gspread.authorize(creds)

sheet_id = '1xqvrDynnWfslrSnOymMtJCrMvmAQBka70L7i8USc5Bs'
service = build('sheets', 'v4', credentials=creds)

body = {
    "requests": [
        # Resize rows
        {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": 0,
                    "dimension": "ROWS",
                    "startIndex": 0,
                    "endIndex": num_rows
                },
                "properties": {
                    "pixelSize": 21
                },
                "fields": "pixelSize"
            }
        },
        # Set text clipping
        {
            "repeatCell": {
                "range": {
                    "sheetId": 0,
                    "startRowIndex": 0,
                    "startColumnIndex": 10,
                    "endColumnIndex": 13
                },
                "cell": {
                    "userEnteredFormat": {
                        "wrapStrategy": "WRAP"
                    }
                },
                "fields": "userEnteredFormat.wrapStrategy"
            }
        }
    ]
}

response = service.spreadsheets().batchUpdate(
    spreadsheetId=sheet_id,
    body=body
).execute()

print("Rows resized and text clipping set successfully.")


Rows resized and text clipping set successfully.


# Port updates to requests gsheets

In [44]:
spreadsheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1IgOnbPhOoCRDBcTf9FIHwP54rHwcqSyKSJTE-XKNnJw/edit?gid=523778578#gid=523778578')
sheet = spreadsheet.get_worksheet(0)
sheet.clear()
set_with_dataframe(sheet, df)
print("Sheet updated successfully.")


Sheet updated successfully.
